<a href="https://colab.research.google.com/github/Osakhra/ITAI2373-NewsBot-Final/blob/main/notebooks/09_Web_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langdetect spacy nltk scikit-learn pyldavis textblob transformers torch sumy sentence-transformers numpy matplotlib seaborn googletrans==4.0.0-rc1 gradio pydantic<2 pydantic-core<2
import nltk
nltk.download('stopwords')


/bin/bash: line 1: 2: No such file or directory


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
!pip install gradio --quiet


In [3]:
!git clone https://github.com/Osakhra/ITAI2373-NewsBot-Final.git
import sys
sys.path.append('/content/ITAI2373-NewsBot-Final/src')


fatal: destination path 'ITAI2373-NewsBot-Final' already exists and is not an empty directory.


In [4]:
import pandas as pd
df = pd.read_csv("ITAI2373-NewsBot-Final/data/processed/news_cleaned.csv")


In [5]:
import sys
sys.path.append('/content/ITAI2373-NewsBot-Final/src')

import pandas as pd
from data_processing.feature_extractor import FeatureExtractor
from analysis.classifier import NewsClassifier
from analysis.sentiment_analyzer import SentimentAnalyzer
from analysis.ner_extractor import NERExtractor
from analysis.topic_modeler import TopicModeler
from language_models.summarizer import Summarizer
from conversation.query_processor import QueryProcessor


In [6]:
from pathlib import Path

# Paths
VEC_PATH = "/content/ITAI2373-NewsBot-Final/data/models/tfidf_vectorizer.pkl"
CLF_PATH = "/content/ITAI2373-NewsBot-Final/data/models/news_classifier_nb.pkl"

# Make sure directory exists
Path("/content/ITAI2373-NewsBot-Final/data/models").mkdir(parents=True, exist_ok=True)

# Train
extractor = FeatureExtractor(max_features=2000, ngram_range=(1,2))
X = extractor.fit_transform(df['clean_content'])
y = df['category']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = NewsClassifier(model_type="nb")
clf.train(X_train, y_train)
clf.evaluate(X_test, y_test)

# Save
extractor.save(VEC_PATH)
clf.save(CLF_PATH)

print("Saved:", VEC_PATH, "and", CLF_PATH)


               precision    recall  f1-score   support

     business       0.97      0.97      0.97        75
entertainment       1.00      0.98      0.99        46
     politics       0.93      0.95      0.94        56
        sport       0.97      1.00      0.98        63
         tech       0.96      0.93      0.95        58

     accuracy                           0.97       298
    macro avg       0.97      0.97      0.97       298
 weighted avg       0.97      0.97      0.97       298

Accuracy: 96.64%
Saved: /content/ITAI2373-NewsBot-Final/data/models/tfidf_vectorizer.pkl and /content/ITAI2373-NewsBot-Final/data/models/news_classifier_nb.pkl


In [7]:
%cd /content/ITAI2373-NewsBot-Final


/content/ITAI2373-NewsBot-Final


In [8]:
df = pd.read_csv("/content/ITAI2373-NewsBot-Final/data/processed/news_cleaned.csv")

extractor = FeatureExtractor()
extractor.load("/content/ITAI2373-NewsBot-Final/data/models/tfidf_vectorizer.pkl")

clf = NewsClassifier(model_type="nb")
clf.load("/content/ITAI2373-NewsBot-Final/data/models/news_classifier_nb.pkl")

sentiment_analyzer = SentimentAnalyzer()
ner_extractor = NERExtractor()
topic_modeler = TopicModeler(n_topics=5, method='lda', max_features=1500)
topic_modeler.fit_transform(df['clean_content'])

summarizer = Summarizer()

qp = QueryProcessor(
    classifier=clf,
    sentiment_analyzer=sentiment_analyzer,
    ner_extractor=ner_extractor,
    topic_modeler=topic_modeler,
    summarizer=summarizer,
    feature_extractor=extractor
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [9]:
import gradio as gr

def chat_interface(query, article_text):
    return qp.process(query, article_text)

demo = gr.Interface(
    fn=chat_interface,
    inputs=[
        gr.Textbox(label="Ask a question (e.g., What is the sentiment?)"),
        gr.Textbox(label="Paste the article content here")
    ],
    outputs="text",
    title="NewsBot 2.0",
    description="Ask questions about news articles. Try: category, sentiment, topic, entities, or summary.",
)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d1dad4bcfee59e28b5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Detected Intent: category
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d1dad4bcfee59e28b5.gradio.live
